### Quiz

. Estimate the memory usage of L-layers Transformer **Encoder** with the dot  
product attention scoring, H-head self-attentions, position-wise networks  
with D-dim output, and maximal sequence length T.  
. Estimate the memory usage of L-layers Transformer **Decoder** with the  
dot product attention scoring, H-head self-attentions, position-wise  
networks with D-dim output, and maximal sequence length T .  
. To increase the expressive power, which parameter should be modified?  

[Transformer Encoder]

Multi-Head Self-Attention:

Query/Key/Value를 위한 프로젝션: 각각 D×D 파라미터를 가지는 세 개의 가중치 행렬 (W_Q, W_K, W_V)이 있으며, 헤드 수 H에 따라 병렬적 수행하나 파라미터 수는 여전히 O(D²) 규모.
결과를 다시 D 차원으로 매핑하는 O(D²) 규모의 출력 프로젝션.
=> 한 층의 MHA 파라미터 수는 대략 4D²에 해당하지만, 여기서는 파라미터보다 메모리 사용량 추정이 핵심.
메모리 관점에서 T 길이의 입력에 대해 Self-Attention은 O(T²) 크기의 어텐션 가중치(각 헤드마다 T×T 크기, H개 헤드 합쳐도 O(H×T²)) 및 각 스텝별로 D 차원 상태를 저장(T×D)한다.
따라서 한 Encoder Layer에서의 메모리 사용량은 대략 O(H×T² + T×D).

Position-wise Feed-Forward Networks(FFN):
일반적으로 2개의 선형 변환(W_1: D→αD, W_2: αD→D; α는 확장 비율, 보통 4 정도)로 이루어져 있으며, 이는 추가로 O(T×D) 정도의 중간 표현을 저장한다.

통합하면, 한 층당 메모리 사용량은 대략 Attention을 통한 O(H×T²) 부분과 FFN을 통한 O(T×D) 부분이 합쳐진다. 파라미터 고정 시, 시퀀스 길이 T가 커지면 T² 항목이 주도적이므로, Encoder L개 층에 대해 총 메모리 사용량은 대략:

O(L×(H×T²+T×D))

In [2]:
def transformer_encoder_memory(L, H, D, T):
    # Encoder memory: O(L * (H*T^2 + T*D))
    return L * (H * (T**2) + T * D)

def transformer_decoder_memory(L, H, D, T):
    # Decoder memory: O(L * (2*H*T^2 + T*D))
    return L * ((2 * H * (T**2)) + T * D)

# 예시 파라미터
L = 6   # 레이어 수
H = 8   # 헤드 수
D = 512 # 차원
T = 128 # 시퀀스 길이

enc_mem = transformer_encoder_memory(L, H, D, T)
dec_mem = transformer_decoder_memory(L, H, D, T)

print("Encoder Memory Estimate:", enc_mem)
print("Decoder Memory Estimate:", dec_mem)

# 표현력 증가: D를 늘려봄
D_new = 1024
enc_mem_expanded = transformer_encoder_memory(L, H, D_new, T)
dec_mem_expanded = transformer_decoder_memory(L, H, D_new, T)

print("Encoder Memory with Increased D:", enc_mem_expanded)
print("Decoder Memory with Increased D:", dec_mem_expanded)

Encoder Memory Estimate: 1179648
Decoder Memory Estimate: 1966080
Encoder Memory with Increased D: 1572864
Decoder Memory with Increased D: 2359296


In [4]:
import torch
import torch.nn as nn

# Transformer 구성 요소 정의
class SimpleMultiHeadSelfAttention(nn.Module):
    def __init__(self, D, H):
        super().__init__()
        self.D = D
        self.H = H
        self.d_k = D // H
        self.W_q = nn.Linear(D, D)
        self.W_k = nn.Linear(D, D)
        self.W_v = nn.Linear(D, D)
        self.W_o = nn.Linear(D, D)
    
    def forward(self, x):
        B, T, D = x.size()
        Q = self.W_q(x).view(B, T, self.H, self.d_k)
        K = self.W_k(x).view(B, T, self.H, self.d_k)
        V = self.W_v(x).view(B, T, self.H, self.d_k)

        attn_weights = torch.matmul(Q, K.transpose(-2, -1)) / (self.d_k ** 0.5)
        attn = torch.matmul(torch.softmax(attn_weights, dim=-1), V)
        attn = attn.view(B, T, D)
        out = self.W_o(attn)
        return out

class PositionWiseFFN(nn.Module):
    def __init__(self, D, expansion=4):
        super().__init__()
        self.fc1 = nn.Linear(D, D*expansion)
        self.fc2 = nn.Linear(D*expansion, D)
    
    def forward(self, x):
        return self.fc2(torch.relu(self.fc1(x)))

class EncoderLayer(nn.Module):
    def __init__(self, D, H, k=4):
        super().__init__()
        self.self_attn = SimpleMultiHeadSelfAttention(D, H)
        self.ffn = PositionWiseFFN(D, expansion=k)
        self.norm1 = nn.LayerNorm(D)
        self.norm2 = nn.LayerNorm(D)
    
    def forward(self, x):
        attn_out = self.self_attn(x)
        x = x + attn_out
        x = self.norm1(x)
        ffn_out = self.ffn(x)
        x = x + ffn_out
        x = self.norm2(x)
        return x

class DecoderLayer(nn.Module):
    def __init__(self, D, H, k=4):
        super().__init__()
        self.self_attn = SimpleMultiHeadSelfAttention(D, H)
        self.ffn = PositionWiseFFN(D, expansion=k)
        self.norm1 = nn.LayerNorm(D)
        self.norm2 = nn.LayerNorm(D)
    
    def forward(self, x):
        attn_out = self.self_attn(x)
        x = x + attn_out
        x = self.norm1(x)
        ffn_out = self.ffn(x)
        x = x + ffn_out
        x = self.norm2(x)
        return x

# 메모리 계산 함수
def calculate_memory(L, H, D, T, k=4):
    # 파라미터 수 계산
    params_per_layer = 4 * D**2 + 2 * k * D**2  # 4D^2 for attention, 2kD^2 for FFN
    total_params = L * params_per_layer

    # 활성화 메모리 계산 (float32 기준 4바이트)
    activations_per_layer = (T**2 * D + T * D**2)  # T^2 D for attention scores, T D^2 for FFN
    total_activations = L * activations_per_layer
    total_activation_bytes = total_activations * 4  # float32

    # 파라미터 메모리 (float32 기준 4바이트)
    total_params_bytes = total_params * 4  # float32

    # 총 메모리 (바이트 단위)
    total_memory_bytes = total_params_bytes + total_activation_bytes

    # 메모리를 GB 단위로 변환
    total_memory_gb = total_memory_bytes / (1024 ** 3)

    return {
        'Total Parameters': total_params,
        'Parameters Memory (GB)': total_params_bytes / (1024 ** 3),
        'Activations Memory (GB)': total_activation_bytes / (1024 ** 3),
        'Total Memory (GB)': total_memory_gb
    }

# 실험용 함수
def test_model_memory(L, H, D, T, k=4):
    encoder_memory = calculate_memory(L, H, D, T, k)
    decoder_memory = calculate_memory(L, H, D, T, k)

    print("=== Encoder-only 모델 메모리 사용량 ===")
    for key, value in encoder_memory.items():
        print(f"{key}: {value:.4f}")
    
    print("\n=== Decoder-only 모델 메모리 사용량 ===")
    for key, value in decoder_memory.items():
        print(f"{key}: {value:.4f}")

# 파라미터 설정
L = 6       # 레이어 수
H = 8       # 헤드 수
D = 512     # 모델 차원
T = 128     # 시퀀스 길이
k = 4       # FFN 확장 비율

test_model_memory(L, H, D, T, k)


=== Encoder-only 모델 메모리 사용량 ===
Total Parameters: 18874368.0000
Parameters Memory (GB): 0.0703
Activations Memory (GB): 0.9375
Total Memory (GB): 1.0078

=== Decoder-only 모델 메모리 사용량 ===
Total Parameters: 18874368.0000
Parameters Memory (GB): 0.0703
Activations Memory (GB): 0.9375
Total Memory (GB): 1.0078
